In [1]:
from utils.general import info, ok, warning, id2file

### Labeled data

In [2]:
import os
import re

second_round_data = [linea.strip().split(';') for linea in  open('second_roud_labels.csv','r').read().splitlines()]
irrelevant_set = set([id_ for id_,label in second_round_data if label=='I'])
relevant_set = set([id_ for id_,label in second_round_data if label=='R'])


# Loading new_data
new_data = [line.split(';') for line in open('new_data.csv').read().splitlines()]
relevant_set = relevant_set.union(set([id_ for id_,label in new_data if label.strip()=='R']))
irrelevant_set = irrelevant_set.union(set([id_ for id_,label in new_data if label.strip()=='I']))

# Loading original data
DP_examples_dirpath = '/home/ec2-user/SageMaker/mariano/notebooks/04. Model of DP/DP-relevant articles/'

first_data = []
for dirpath, dirnames, filenames in os.walk(DP_examples_dirpath):
    for filename in filenames:
        content = open(os.path.join(dirpath,filename),'r').read()
        ids = re.findall('/docview/([^/]*)/',content)
        relevant_set = relevant_set.union(set(ids))
    
# articles containg DP and Canada from that period, that were not deteted by Serperi
GM_dp_dirpath = '/home/ec2-user/SageMaker/data/GM_DP_and_Canada1945_1967/'

files = os.listdir(GM_dp_dirpath)

irrelevant_set = irrelevant_set.union([file_[:-4] for file_ in files if file_[:-4] not in relevant_set and file_.endswith('.xml')])

not_found=[]
for id_ in list(relevant_set)+list(irrelevant_set):
    if id2file(id_) is None:
        not_found.append(id_)
print(f'Not found: {not_found}')
for id_ in not_found:
    relevant_set = relevant_set.difference(set(not_found))
    irrelevant_set = irrelevant_set.difference(set(not_found))
    
info(f'len(relevant_set)   = {len(relevant_set)}')
info(f'len(irrelevant_set) = {len(irrelevant_set)}')


Not found: ['1222379804', '1136691129', '2122279956', '1238204962', '1411697642', '1238440920', '2459666609', '1242257052', '2459964104', '1151348424', '2122281371', '1239753620', '1143160388']
2022-03-17 12:50:50.756200 [ INFO  ] len(relevant_set)   = 581
2022-03-17 12:50:50.756332 [ INFO  ] len(irrelevant_set) = 6523


### GloVe 300

In [4]:
import numpy as np
from utils.tdmstudio import TDMStudio
import spacy
nlp = spacy.load('en_core_web_lg',disable=['textcat', 'ner', 'lemmatizer','parser','tagger'])

X = np.zeros(shape=(len(relevant_set)+len(irrelevant_set),300))

for idx,id_ in enumerate(list(relevant_set)+list(irrelevant_set)):
    title,text = TDMStudio.get_title_and_text(id2file(id_))
    X[idx,:] = nlp(f'{title}. {text}').vector
    del(title,text)
# corpus = [TDMStudio.get_title_and_text(id2file(id_)) for id_ in list(relevant_set)+list(irrelevant_set)]
# corpus = [f'{title}. {text}' for title,text in corpus]
X


array([[-0.02825912,  0.0889851 , -0.09836917, ..., -0.02832093,
        -0.00369143,  0.00478626],
       [ 0.05525575,  0.11047117, -0.01124459, ..., -0.02380759,
        -0.0377602 ,  0.06344097],
       [ 0.02058255,  0.11505025, -0.0824295 , ..., -0.02079856,
         0.01385511,  0.04198706],
       ...,
       [ 0.00234893,  0.13833135, -0.02347194, ..., -0.06200312,
        -0.03389218,  0.05006967],
       [-0.04666149,  0.31146714,  0.13649581, ..., -0.08591507,
        -0.01002735, -0.00262582],
       [ 0.02852197,  0.12607905, -0.02006903, ..., -0.04940524,
        -0.01372787,  0.05615823]])

### GloVe 600

In [5]:
import numpy as np
from utils.tdmstudio import TDMStudio
import spacy
nlp = spacy.load('en_core_web_lg',disable=['textcat', 'ner', 'lemmatizer','parser','tagger'])

X = np.zeros(shape=(len(relevant_set)+len(irrelevant_set),600))

for idx,id_ in enumerate(list(relevant_set)+list(irrelevant_set)):
    title,text = TDMStudio.get_title_and_text(id2file(id_))
    X[idx,:300]= nlp(title).vector
    X[idx,300:]= nlp(text).vector
#     X[idx,:] = nlp(f'{title}. {text}').vector
    del(title,text)
# corpus = [TDMStudio.get_title_and_text(id2file(id_)) for id_ in list(relevant_set)+list(irrelevant_set)]
# corpus = [f'{title}. {text}' for title,text in corpus]
X


array([[-0.11897325,  0.42263442, -0.01137875, ..., -0.02408694,
         0.01733126,  0.04512096],
       [-0.08863501,  0.1368888 ,  0.02039914, ..., -0.02852704,
        -0.03971205,  0.09252498],
       [-0.0251535 ,  0.16785686,  0.02735153, ...,  0.01089294,
        -0.0502997 ,  0.08150491],
       ...,
       [-0.2283535 ,  0.1759377 , -0.20194285, ..., -0.02903259,
        -0.04929906,  0.06473676],
       [-0.24161184,  0.18289351, -0.16367717, ..., -0.0387418 ,
        -0.01256677,  0.05979146],
       [-0.28133681,  0.17674434, -0.28741285, ..., -0.06954952,
         0.02872961,  0.00753791]])

In [6]:
print('h')

h


In [6]:
import numpy as np
y = np.zeros(shape=(X.shape[0],))
y[:len(relevant_set)]=1
y

array([1., 1., 1., ..., 0., 0., 0.])

In [11]:
import pandas as pd

from sklearn.model_selection import cross_validate
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
parameters = { 'C':[0.1, 0.5, 1, 3,5,7,9,11,15,20], 'degree':[1,2,3,4,7]}
svc = SVC(kernel='poly')


clf = GridSearchCV(svc, parameters,scoring='f1', cv=5,verbose=4)

results = clf.fit(X,y)
pd.DataFrame(results.cv_results_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END ...................C=0.1, degree=1;, score=0.784 total time=   2.8s
[CV 2/5] END ...................C=0.1, degree=1;, score=0.780 total time=   2.4s
[CV 3/5] END ...................C=0.1, degree=1;, score=0.686 total time=   2.5s
[CV 4/5] END ...................C=0.1, degree=1;, score=0.802 total time=   2.6s
[CV 5/5] END ...................C=0.1, degree=1;, score=0.700 total time=   2.4s
[CV 1/5] END ...................C=0.1, degree=2;, score=0.793 total time=   2.3s
[CV 2/5] END ...................C=0.1, degree=2;, score=0.795 total time=   2.4s
[CV 3/5] END ...................C=0.1, degree=2;, score=0.683 total time=   2.4s
[CV 4/5] END ...................C=0.1, degree=2;, score=0.803 total time=   2.4s
[CV 5/5] END ...................C=0.1, degree=2;, score=0.704 total time=   2.3s
[CV 1/5] END ...................C=0.1, degree=3;, score=0.784 total time=   2.3s
[CV 2/5] END ...................C=0.1, degree=3

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_degree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,2.267272,0.121515,0.270245,0.006866,0.1,1,"{'C': 0.1, 'degree': 1}",0.784483,0.779528,0.685714,0.801688,0.700361,0.750355,0.047600,28
1,2.100191,0.073384,0.254936,0.007605,0.1,2,"{'C': 0.1, 'degree': 2}",0.793103,0.795181,0.683128,0.803347,0.703704,0.755693,0.051378,18
2,1.982721,0.038017,0.247717,0.006564,0.1,3,"{'C': 0.1, 'degree': 3}",0.784483,0.803279,0.669492,0.806867,0.716981,0.756220,0.054111,16
3,2.080938,0.070523,0.249660,0.004548,0.1,4,"{'C': 0.1, 'degree': 4}",0.784141,0.796610,0.675325,0.812500,0.714829,0.756681,0.052602,14
4,2.205574,0.044684,0.261818,0.005415,0.1,7,"{'C': 0.1, 'degree': 7}",0.704762,0.735849,0.660550,0.720000,0.700000,0.704232,0.025197,50
5,1.876600,0.046729,0.215962,0.005857,0.5,1,"{'C': 0.5, 'degree': 1}",0.796537,0.786611,0.683983,0.808511,0.708661,0.756860,0.050520,12
6,1.831045,0.049187,0.207637,0.006523,0.5,2,"{'C': 0.5, 'degree': 2}",0.775330,0.773913,0.714286,0.826087,0.730159,0.763955,0.039238,5
7,1.856144,0.049782,0.208722,0.005181,0.5,3,"{'C': 0.5, 'degree': 3}",0.791111,0.775330,0.713656,0.817778,0.738956,0.767366,0.037042,3
8,1.948487,0.054580,0.214152,0.006368,0.5,4,"{'C': 0.5, 'degree': 4}",0.786026,0.778761,0.711111,0.819820,0.751020,0.769348,0.036446,2
9,2.081816,0.082311,0.225817,0.007120,0.5,7,"{'C': 0.5, 'degree': 7}",0.748858,0.784141,0.669643,0.781818,0.730290,0.742950,0.041896,39


In [11]:
pd.DataFrame(results.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.775025,0.084095,0.239847,0.006633,3,{'C': 3},0.820717,0.850000,0.864407,0.801653,0.812227,0.829801,0.023619,6
1,0.689681,0.030092,0.226961,0.010157,5,{'C': 5},0.824000,0.843882,0.868085,0.808333,0.829694,0.834799,0.020167,5
2,0.691777,0.049221,0.221342,0.010067,7,{'C': 7},0.827309,0.847458,0.869198,0.822034,0.838428,0.840885,0.016677,1
3,0.702308,0.029808,0.217649,0.009928,9,{'C': 9},0.830645,0.837607,0.863071,0.813559,0.838428,0.836662,0.015946,4
4,0.601003,0.019660,0.202838,0.003315,12,{'C': 12},0.827309,0.836207,0.865546,0.813559,0.848485,0.838221,0.017795,2
5,0.617704,0.026087,0.194587,0.003865,15,{'C': 15},0.822581,0.824034,0.875000,0.820513,0.844828,0.837391,0.020748,3
